In [1]:
import torch
import numpy as np
from torchvision import datasets, transforms
from matplotlib import pyplot as plt
from metric.niqe import niqe
from metric.fid import calculate_fid
from skimage.transform import rescale, resize
from imageio import imread, imwrite
from collections import defaultdict
from tqdm import tqdm
from glob import glob

In [2]:
imgs = np.moveaxis(np.array([imread(f'../stargan/stargan_celeba/results/{i + 1}-images.jpg') for i in range(100)]).reshape(100, 64, -1, 64,3), 2, 1)
ori = np.array([i[0] for i in imgs])
# rec = np.array([i[1] for i in imgs])
# feat = np.array([i[2:] for i in imgs])

# for i, img in enumerate(ori):
#     imwrite(f'cgan_result/ori/{i}.png', img)
    
# for i, img in enumerate(rec):
#     imwrite(f'cgan_result/rec/{i}.png', img)
    
# for i, img in enumerate(feat):
#     for j, fimg in enumerate(img):
#         imwrite(f'cgan_result/f{j+1}/{i}.png', fimg)
        
rec = np.array([imread(f'cvae_samples/reconstruct/{i}.png') for i in range(100)])[:, :, :, :3]
feat = np.array([[imread(f'cvae_samples/fake_attr/{i}_{j}.png') for j in range(5)] for i in range(100)])[:, :, :, :, :3]

for i, img in enumerate(ori):
    imwrite(f'cvae_result/ori/{i}.png', img)
    
for i, img in enumerate(rec):
    imwrite(f'cvae_result/rec/{i}.png', img)
    
for i, img in enumerate(feat):
    for j, fimg in enumerate(img):
        imwrite(f'cvae_result/f{j+1}/{i}.png', fimg)

In [3]:
n, h, _, w, c = imgs.shape
n_feat = feat.shape[1]
score_dic = defaultdict(list)

score_dic[f'fid_rec'].append(calculate_fid(ori, rec, False, 1))
for j in range(n_feat):
    score_dic[f'fid_{j}'].append(calculate_fid(ori, feat[:, j], False, 1))
    for i in range(n):
        if j == 0: 
            score_dic[f'rec'].append(((ori[i] - rec[i])**2 / 255.**2).mean())
            
            s = np.mean(ori[i], axis=2)
            s.resize(200, 200)
            score_dic[f'niqe_ori'].append(niqe(s))

            niqe_score = 0
            for k in range(3):
                s = np.copy(rec[i, :, :, k])
                s.resize(200, 200)
                niqe_score += niqe(s)
            score_dic[f'niqe_rec'].append(niqe_score / 3.)
        
        s = np.mean(feat[i, j], axis=2)
        s.resize(200, 200)
        score_dic[f'niqe_{j}'].append(niqe(s))

In [4]:
print('Reconstruction Loss: {:.4f}'.format(np.mean(score_dic['rec'])))

Reconstruction Loss: 0.0012


In [5]:
print('NIQE Ori: {:.4f}'.format(np.mean(score_dic['niqe_ori'])))
print('NIQE Rec: {:.4f}'.format(np.mean(score_dic['niqe_rec'])))
print('FID Rec: {:.4f}'.format(np.mean(score_dic['fid_rec'])))

NIQE Ori: 26.9139
NIQE Rec: 26.8681
FID Rec: 110.9635


In [6]:
for j in range(n_feat):
    print('Feature:{}, FID: {:.4f}, NIQE: {:.4f}'.format(j, np.mean(score_dic[f'fid_{j}']), np.mean(score_dic[f'niqe_{j}'])))

Feature:0, FID: 110.3758, NIQE: 26.8451
Feature:1, FID: 121.7876, NIQE: 26.8723
Feature:2, FID: 113.3765, NIQE: 26.8424
Feature:3, FID: 116.9071, NIQE: 26.8434
Feature:4, FID: 111.5689, NIQE: 26.8777
